In [ ]:
! pip install kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
! pip install http.client

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 234.2/234.2 KB 13.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for http.client: filename=http_client-0.1.22-py3-none-any.whl size=2207 sha256=865348eb42c8a2f07787c38d051decbb97233502d8aebdd36fc2bcdc43d3b7fe
  Stored in directory: /root/.cache/pip/wheels/92/ca/0c/9069b0e24e1dc035db1051922e7472c3f14ce61a0f6022e982
  Created wheel for pycurl: filename=pycurl-7.45.2-cp39-cp39-linux_x86_64.whl size=334189 sha256=db59be61fa8c50758058db768cda66a5c671569be1a047f28ef1ebf37b58a561
  Stored in directory: /root/.cache/pip/wheels/23/b0/37/d2c3211ee738adfb8ec6b6e10aa00e78ebc4de363f862a12c5
Successfully built http.client pycurl


In [ ]:
! pip install urlparse4 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.4/158.4 KB 14.2 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [1]:
! mkdir ~/.kaggle

In [3]:
! cp kaggle.json ~/.kaggle/

In [4]:
! chmod 600 ~/.kaggle/kaggle.json

In [5]:
! kaggle datasets download -d thedevastator/explore-reddit-s-funny-subreddit-analyze-communi

100% 90.9k/90.9k [00:00<00:00, 661kB/s]
100% 90.9k/90.9k [00:00<00:00, 659kB/s]


In [6]:
!unzip /content/explore-reddit-s-funny-subreddit-analyze-communi.zip -d /content/explore-reddit-s-funny-subreddit-analyze-communi

Archive:  /content/explore-reddit-s-funny-subreddit-analyze-communi.zip
  inflating: /content/explore-reddit-s-funny-subreddit-analyze-communi/funny.csv  


In [ ]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import VGG16
from tensorflow.keras import layers


In [ ]:

#Old
# Load the data from CSV file
data = pd.read_csv('/content/explore-reddit-s-funny-subreddit-analyze-communi/funny.csv')

# Split the dataset into training and validation sets
train_data, val_data = train_test_split(data, test_size=0.2, random_state=42)

# Create training and validation dataframes
train_df = pd.DataFrame({
    'url': train_data['url'].astype(str), # replace 'url_column_name' with the actual column name in your CSV file
    'score': train_data['score'].astype(str) # replace 'label_column_name' with the actual column name in your CSV file
})

val_df = pd.DataFrame({
    'url': val_data['url'].astype(str), # replace 'url_column_name' with the actual column name in your CSV file
    'score': val_data['score'].astype(str) # replace 'label_column_name' with the actual column name in your CSV file
})

# Define the data generator for training set
train_datagen = ImageDataGenerator(rescale=1./255)

# Load the training images and labels from URLs
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    x_col="url", # specify the column name containing the URLs
    y_col="score", # specify the column name containing the labels
    target_size=(224, 224),
    batch_size=32,
    class_mode="binary"
)

# Define the data generator for validation set
val_datagen = ImageDataGenerator(rescale=1./255)

# Load the validation images and labels from URLs
val_generator = val_datagen.flow_from_dataframe(
    dataframe=val_df,
    x_col="url", # specify the column name containing the URLs
    y_col="score", # specify the column name containing the labels
    target_size=(224, 224),
    batch_size=32,
    class_mode="binary"
)

# Load pre-trained VGG16 model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Create a new model for binary classification
x = base_model.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(1024, activation='relu')(x)
predictions = layers.Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the layers of the pre-trained model
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer=Adam(lr=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(train_generator, epochs=10, validation_data=val_generator)


FileNotFoundError: ignored

In [ ]:
#New
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import requests
import http.client
from urlparse import urllib.parse 

# a function to check the response status code of an image URL in the URL column 
#def check_image_url(url):
  #  response = requests.get(url)
   # return response.status_code
import csv
with open('/content/explore-reddit-s-funny-subreddit-analyze-communi/funny.csv') as fobj:
    reader = csv.DictReader(fobj)
    for row in reader:
        valid = checkUrl(row['url'])
        print('%s is %svalid' % (row['url'], '' if valid else 'in'))
# Loading the data
data = pd.read_csv('/content/explore-reddit-s-funny-subreddit-analyze-communi/funny.csv')
data['url'] = data['url'].fillna('https://example.com/image.jpg')

data = data.dropna(subset=['url'])
# Remove rows with invalid image URLs
data['status_code'] = data['url'].apply(check_image_url)
data = data[data['status_code'] == 200]

# Define the labels
data['funny'] = np.where(data['score'] >= 1000, 1, 0)
data['funny'] = data['funny'].astype(str)

# Define the data generator
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2, format=['jpg', 'png', 'gif'])
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

# Load the images and labels from URLs
generator = datagen.flow_from_dataframe(
    dataframe=data,
    x_col='url',
    y_col='funny',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='training'
)

val_generator = datagen.flow_from_dataframe(
    dataframe=data,
    x_col='url',
    y_col='funny',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='validation'
)

# Load the pre-trained VGG16 model transfer learning
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Create a new model for binary classification
x = base_model.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(1024, activation='relu')(x)
predictions = layers.Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the layers of the pre-trained model
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer=Adam(lr=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(generator, epochs=10, validation_data=val_generator)


ModuleNotFoundError: ignored

In [7]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import requests

# Define a function to check the response status code of an image URL
def check_image_url(url):
    response = requests.get(url)
    return response.status_code

# Load the data
data = pd.read_csv('/content/explore-reddit-s-funny-subreddit-analyze-communi/funny.csv')
data['url'] = data['url'].fillna('https://example.com/image.jpg')

# Remove rows with invalid image URLs
data['status_code'] = data['url'].apply(check_image_url)
data = data[data['status_code'] == 200]

# Define the labels
data['funny'] = np.where(data['score'] >= 1000, 1, 0)
data['funny'] = data['funny'].astype(str)

# Define the data generator
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2, valid_formats=['jpg', 'png', 'gif'])

# Load the images and labels from URLs
generator = datagen.flow_from_dataframe(
    dataframe=data,
    x_col='url',
    y_col='funny',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='training'
)

val_generator = datagen.flow_from_dataframe(
    dataframe=data,
    x_col='url',
    y_col='funny',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='validation'
)

# Load the pre-trained VGG16 model transfer learning
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Create a new model for binary classification
x = base_model.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(1024, activation='relu')(x)
predictions = layers.Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the layers of the pre-trained model
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(generator, epochs=10, validation_data=val_generator)


KeyboardInterrupt: ignored

Test 4: 

In [8]:
import pandas as pd
import numpy as np
import requests
from PIL import Image
import io
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam


In [ ]:
# Load the dataset
dataset_path = '/content/explore-reddit-s-funny-subreddit-analyze-communi/funny.csv'
data = pd.read_csv(dataset_path)

# Define a function to download an image from a URL
def download_image(url):
    try:
        response = requests.get(url, timeout=5)
        img = Image.open(io.BytesIO(response.content))
        return img
    except:
        return None

# Download the images
images = []
scores = []

for index, row in data.iterrows():
    url = row['url']
    score = row['score']
    img = download_image(url)

    if img is not None:
        images.append(img)
        scores.append(score)

# Preprocess the images
input_size = (224, 224)
threshold = 1000
X = np.array([np.array(img.convert('RGB').resize(input_size)) for img in images])
y = np.array(['fun' if score > threshold else 'not_fun' for score in scores])

# Encode the labels
encoder = LabelEncoder()
y = encoder.fit_transform(y)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
# Load the VGG16 model with pre-trained ImageNet weights
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the layers of the base model
for layer in base_model.layers:
    layer.trainable = False

# Add custom layers
x = Flatten()(base_model.output)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(1, activation='sigmoid')(x)

# Create the final model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=32)

# Save the model
model.save('fun_or_not_fun_model.h5')
